In [ ]:
import warnings
warnings.filterwarnings('ignore')

from astropy.table import QTable, join, vstack
import astropy.units as au
import numpy as np
from glob import glob
import os

In [ ]:
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'
dendro_dir = '%s/data_hstha_nebulae_catalogue/' %(root_dir)

tables = []
for root, dirs, files in os.walk(dendro_dir):
    for file in files:
        if file.endswith("props_all_cluster.fits"):
            if 'dendro' in root: 
                continue
            print(os.path.join(root, file))
            tables += [QTable.read(os.path.join(root, file))]

In [ ]:
# Join tables
table = vstack(tables)

# Order by galaxy names and region IDs from MUSE
gal_name = table['gal_name']
table.remove_column('gal_name')
table.add_column(gal_name, index=0)
table.sort(['gal_name', 'region_ID'])

# # Remove masked values
# mask = table['_idx'].mask
# table_masked = table[~mask]

In [ ]:
table.write(dendro_dir+'/all/nebulae_catalogue_wclusters.fits', overwrite=True)

In [ ]:
mask = table['gal_name'] == 'NGC4303'
table_masked = table[mask]

for region_ID, reg_id in zip(table_masked['region_ID'], table_masked['reg_id']):
    print(region_ID, reg_id)

In [ ]:
table

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('paper')

table_masked_ = table[table['reg_dolflux_Age_MinChiSq']<10]

fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)

ax.scatter(table_masked_['mean_sigma_pc'], 
            table_masked_['HA6562_LUMINOSITY_HST']/table_masked_['mean_sigma_pc']**2,
            c=table_masked_['reg_dolflux_Age_MinChiSq'],
            s=np.log10(table_masked_['reg_dolflux_Mass_MinChiSq']))

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlabel('Radius (pc)')
ax.set_ylabel('Luminosity (erg/s)')
ax.grid(alpha=0.3, linestyle=':')

fig.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('paper')

table_masked_ = table[table['reg_dolflux_Age_MinChiSq']<10]

fig = plt.figure(figsize=(8,3))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.scatter(table_masked_['reg_dolflux_Age_MinChiSq'], 
            table_masked_['HA6562_LUMINOSITY_HST']/table_masked_['mean_sigma_pc']**2)

ax2.scatter(table_masked_['reg_dolflux_Mass_MinChiSq'], 
            table_masked_['HA6562_LUMINOSITY_HST']/table_masked_['mean_sigma_pc']**2)

for ax in [ax1, ax2]:
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.grid(alpha=0.3, linestyle=':')
    
# ax.set_xlabel('Radius (pc)')
# ax.set_ylabel('Luminosity (erg/s)')

fig.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(12,3))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)

bins = np.linspace(0,50,50)
ax1.hist(table['reg_dolflux_Age_MinChiSq'], alpha=0.3, fc='C0', bins=bins)
ax1.hist(table['reg_dolflux_Age_MinChiSq'], fc='none', ec='C0', lw=1.5, bins=bins)
ax1.set_ylabel('#')
ax1.set_xlabel('Cluster age, Myr')
ax1.grid(alpha=0.3, linestyle=':')
# ax1.set_xlim(0,20)

bins = np.linspace(2,9,50)
ax2.hist(np.log10(table['reg_dolflux_Mass_MinChiSq']), alpha=0.3, fc='C1', bins=bins)
ax2.hist(np.log10(table['reg_dolflux_Mass_MinChiSq']), fc='none', ec='C1', lw=1.5, bins=bins)
ax2.set_ylabel('#')
ax2.set_xlabel('Cluster mass, log(Msun)')
ax2.grid(alpha=0.3, linestyle=':')

bins = np.linspace(0,2,50)
ax3.hist(table['reg_dolflux_Ebv_MinChiSq'], alpha=0.3, fc='C2', bins=bins)
ax3.hist(table['reg_dolflux_Ebv_MinChiSq'], fc='none', ec='C2', bins=bins)
ax3.set_ylabel('#')
ax3.set_xlabel('EBV (mag)')
ax3.grid(alpha=0.3, linestyle=':')

fig.tight_layout()